In [1]:
import json
import csv
import tweepy
import re
from datetime import date,datetime,timedelta

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import yfinance as yf

In [2]:
class VaderSentimentService:
    def __init__(self):
        self.vader = SentimentIntensityAnalyzer()
    
    def __call__(self, text):
        return self.vader.polarity_scores(text)['compound']


## 1. Twitter API:

In [9]:
vader = VaderSentimentService()
import tweepy


class TwitterScraper:
    def __init__(self):
        self.consumer_key = 'jHtPtALPjPLqpSiwZJqTAbWEZ'
        self.consumer_secret = 'ETFeqC0WDvVnV0Vd497FkzsYCRy2mf1UJHTH4SX64qaXVEtlwX'
        self.access_token = '1265371572942102528-L4mxBbGrnpGIBX0l9Dnu85TZj2f3xg'
        self.access_token_secret = 'dnCdwYftc47v4zlDUsbo9ap2RlD4B3tqnU9DBrQS41yAS'
        self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
        self.auth.set_access_token(self.access_token, self.access_token_secret)

    def __call__(self, hashtag_phrase, start_date=date.today(), end_date=date.today()+timedelta(days=1)):
        format_hashtag = '\$' + hashtag_phrase

        # api = tweepy.API(self.auth)
        # results = api.search_tweets(q=format_hashtag)
        api = tweepy.Client(
            bearer_token='AAAAAAAAAAAAAAAAAAAAAHjLGAEAAAAAAqhN%2Bz0VZ1DDDuiPGL5dNidSX7U%3DFcr9bkGHz4Zh6oEXMaQ8fu12xyrVAVh3lV1ae3cwFOXOC7smSs')

        scraped_tweets = api.search_recent_tweets(
            query=format_hashtag,
            expansions = ['author_id'],
            tweet_fields = ['author_id','created_at','text','lang'],
            user_fields = ['name','username','verified'],
        )

        augmented_tweets = []
        for tweet in scraped_tweets.data:
            cleaned_text = self._clean_tweets(tweet.data['text'])
            augmented_tweets.append(
                {
                    'created_at': tweet.data['created_at'],
                    'text': cleaned_text,
                    'sentiment': vader(cleaned_text)
                }
            )

        return augmented_tweets
        
    def _remove_pattern(self, input_txt, pattern):
        r = re.findall(pattern, input_txt)
        for i in r:
            input_txt = re.sub(i, '', input_txt)       
        return input_txt
        
    def _clean_tweets(self, tweets):
        #remove twitter Return handles (RT @xxx:)
        tweets = np.vectorize(self._remove_pattern)(tweets, "RT @[\w]*:") 
        #remove twitter handles (@xxx)
        tweets = np.vectorize(self._remove_pattern)(tweets, "@[\w]*")
        #remove URL links (httpxxx)
        tweets = np.vectorize(self._remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
        #remove special characters, numbers, punctuations (except for #)
        tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
        return str(tweets)
        

In [10]:
twitter = TwitterScraper()
res = twitter('AAPL')

In [11]:
res[0]

{'created_at': '2022-12-08T05:06:20.000Z',
 'text': 'Over 80k+ people. Join now\nDiscord group👇\n\n\n\n$CGM $APT $JAKK $DYNT $SPV $LMND $GNUS $GRAF  $CLSN $CHFS $MARK $SHLO $NIO $DLPN $WKHS $CARV $SPY $SOLO $W $JAGX $INO  $BYFC $TSLA $SPY $BOXL $AAPL $AYRO $BNGO $PHUN ',
 'sentiment': -0.128}

In [6]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)       
    return input_txt
    
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    return tweets

def get_tweets(hashtag_phrase):
    format_hashtag = '$'+hashtag_phrase
    start_date = date.today()
    end_date = date.today()+timedelta(days=1)
    # os.environ["consumer_key"] = "psxikKIcDvu19SDl1qbTycOHY"
    # os.environ["consumer_secret"] = "H2FV5HL4UyuzVUtANd0lo3HpsNzo1woDMuwlabVyv2E7g44SDb"
    # os.environ["twitter_access_token"] = "1265371572942102528-XZyshdLemub7C7hrx51dyFrYCBCmvU"
    # os.environ["twitter_access_secret"] = "CK4hmNukUYBhzr3CAtO3gvGFx7Ahr9TW3vZMjW6pl8yVd"

    # os.environ["consumer_key"] = "4BSOA2XKS8vucCHyBjy502Aw8"
    # os.environ["consumer_secret"] = "l27Zsdh9X9oGGff25gJ4PX9zN6ZnjYKonu2zISu17jsQlO5Dkb"
    # os.environ["twitter_access_token"] = "1265371572942102528-IsosHWjrXRDKHaqQFAbSPuM2FyH41k"
    # os.environ["twitter_access_secret"] = "pVjYFAx8pDCBnJE48NxsA4KB6g4eNMw39TOWcTciLOB5u"

    os.environ["consumer_key"] = "dQXGlGv8YFtPrZvQwcCnvbged"
    os.environ["consumer_secret"] = "ouKbQUg1dGubyKbp7DrNo45Qdv3nNzd7MyuvCgIKha0vpuNbDA"
    os.environ["twitter_access_token"] = "1265371572942102528-Jmu9hvd4yBep0KwV9U5mHFUnyUi9JV"
    os.environ["twitter_access_secret"] = "CYW3FRkyXJnSzRrHoN9FcLlBdSRHfA7GWJK1PMT7Q1S7P"

    # os.environ["consumer_key"] = "ljU1UWBCC0YNKlO9pwm1TshUc"
    # os.environ["consumer_secret"] = "w5CDP6fNeNQhvDs075KiZQEIWI7VY1Z8BxiDc5kUAHTsXzOhCY"
    # os.environ["twitter_access_token"] = "1309241898419322880-hBG39tjNql0FcHrjOMYg2qTN3TEtnw"
    # os.environ["twitter_access_secret"] = "p0UOsVpTB9hu15R3YBMbWGw2zDpJjkUCmK0YadzA894ZF"
    consumer_key = os.environ['consumer_key']
    consumer_secret = os.environ['consumer_secret']
    access_token = os.environ['twitter_access_token']
    access_token_secret = os.environ['twitter_access_secret']

    auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
    auth.set_access_token(access_token,access_token_secret)

    api = tweepy.API(auth)
   
    twitter_posts = pd.DataFrame(columns=['timestamp', 'tweet_text', 'followers_count'])
    timestamp=[]
    tweets=[]
    follow_count=[]
    # while True:
    #     try:
    for tweet in tweepy.Cursor(api.search, q=format_hashtag+' -filter:retweets', lang="en", tweet_mode='extended',since=start_date, until=end_date).items():
        timestamp.append(tweet.created_at)
        tweets.append(tweet.full_text.replace('\n',' ').encode('utf-8'))
        follow_count.append(tweet.user.followers_count)
        # except tweepy.TweepError:
        #     break
        # except StopIteration:
        #     break
    twitter_posts['timestamp']=timestamp
    twitter_posts['tweet_text']=tweets
    twitter_posts['followers_count']=follow_count
    twitter_posts['tweet_text']=twitter_posts['tweet_text'].str.decode("utf-8")
    twitter_posts['scaled_followers_count'] =twitter_posts['followers_count']/twitter_posts['followers_count'].max()

    vader = SentimentIntensityAnalyzer()
    twitter_posts['tweet_text'] = clean_tweets(twitter_posts['tweet_text'])
    # dataframe.reset_index(drop=False,inplace=True)
    scores = twitter_posts['tweet_text'].apply(vader.polarity_scores).tolist()
    scores_df = pd.DataFrame(scores)
    df = twitter_posts.join(scores_df, rsuffix='_right')
    df['compound'] = df['compound']*(twitter_posts['scaled_followers_count']+1)

    df.to_csv('~/LighthouseLabs-Final/' + hashtag_phrase + '_' + (datetime.today().strftime('%Y-%m-%d')) + '.csv')
    return df

In [7]:
get_tweets('TSLA')

,timestamp,tweet_text,followers_count,scaled_followers_count,neg,neu,pos,compound
0,2020-10-13 14:10:50,Aggressive dip buying on $FEAC. New 52-week hi...,1303,0.001773,0.032,0.891,0.077,0.511505
1,2020-10-13 14:10:46,"$HTSC, Battery metal, EV play OVERSOLD. Last t...",1367,0.001860,0.000,0.953,0.047,0.340633
2,2020-10-13 14:10:44,$TSLA also setting up another tighter pause pi...,152,0.000207,0.000,1.000,0.000,0.000000
3,2020-10-13 14:10:23,$TSLA $450c 10/16 and $500c 10/23. This thing ...,101,0.000137,0.070,0.930,0.000,-0.273238
4,2020-10-13 14:10:14,Today at 10:10 AM $TSLA holding up nice around...,1349,0.001836,0.000,0.763,0.237,0.422274
...,...,...,...,...,...,...,...,...
1408,2020-10-13 00:00:45,then there is the battery life/storage. this...,9941,0.013529,0.000,1.000,0.000,0.000000
1409,2020-10-13 00:00:31,"$TSLA and $SPCE, the frauds you promote, also?",200,0.000272,0.277,0.504,0.218,-0.177948
1410,2020-10-13 00:00:21,. tell of big news. New castle takeover comple...,8,0.000011,0.000,0.950,0.050,0.296003
1411,2020-10-13 00:00:01,Real trade for profit and excellent chatroom.....,13,0.000018,0.000,0.852,0.148,0.765014


## 2. News Headlines:

In [ ]:
class NewsHeadlineScraper:
    def __init__(self):
        self.finviz_url = 'https://finviz.com/quote.ashx?t='
    
    def __call__(self, ticker_symbols):
        ticker_news = {}
        for ticker in ticker_symbols:
            url = self.finwiz_url + ticker
            req = Request(
                url=url,
                headers={'user-agent': 'my-app/0.0.1'},
            )
            
            response = urlopen(req)    
            # Read the contents of the file into 'html'
            html = BeautifulSoup(response)
            # Find 'news-table' in the Soup and load it into 'news_table'
            news_table = html.find(id='news-table')
            # Add the table to our dictionary
            ticker_news[ticker] = news_table

        parsed_news = self._parse_news(ticker_news)

        for news in parsed_news:
            news.update(vader(news['headline']))

        return parsed_news
    
    def _parse_news(self, ticker_news):
        parsed_news = []
        # Iterate through the news
        for file_name, news_table in ticker_news.items():
            # Iterate through all tr tags in 'news_table'
            for x in news_table.findAll('tr'):
                # read the text from each tr tag into text
                # get text from a only
                headline = x.a.get_text() 
                # splite text in the td tag into a list 
                date_scrape = x.td.text.split()
                # if the length of 'date_scrape' is 1, load 'time' as the only element

                if len(date_scrape) == 1:
                    time = date_scrape[0]
                    
                # else load 'date' as the 1st element and 'time' as the second    
                else:
                    date = date_scrape[0]
                    time = date_scrape[1]
                # Extract the ticker from the file name, get the string up to the 1st '_'  
                ticker = file_name.split('_')[0]
                
                # Append ticker, date, time and headline as a list to the 'parsed_news' list
                parsed_news.append(
                    {
                        'ticker': ticker,
                        'date': date,
                        'time': time,
                        'headline': headline,
                    }
                )


In [11]:
def get_news(ticker_code):
    # 1. Define URL:
    finwiz_url = 'https://finviz.com/quote.ashx?t='
    # 2. Requesting data:
    news_tables = {}
    tickers = [ticker_code]
    for ticker in tickers:
        url = finwiz_url + ticker
        req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
        response = urlopen(req)    
        # Read the contents of the file into 'html'
        html = BeautifulSoup(response)
        # Find 'news-table' in the Soup and load it into 'news_table'
        news_table = html.find(id='news-table')
        # Add the table to our dictionary
        news_tables[ticker] = news_table
    #3. Parsing news:
    parsed_news = []
    # Iterate through the news
    for file_name, news_table in news_tables.items():
        # Iterate through all tr tags in 'news_table'
        for x in news_table.findAll('tr'):
            # read the text from each tr tag into text
            # get text from a only
            text = x.a.get_text() 
            # splite text in the td tag into a list 
            date_scrape = x.td.text.split()
            # if the length of 'date_scrape' is 1, load 'time' as the only element

            if len(date_scrape) == 1:
                time = date_scrape[0]
                
            # else load 'date' as the 1st element and 'time' as the second    
            else:
                date = date_scrape[0]
                time = date_scrape[1]
            # Extract the ticker from the file name, get the string up to the 1st '_'  
            ticker = file_name.split('_')[0]
            
            # Append ticker, date, time and headline as a list to the 'parsed_news' list
            parsed_news.append([ticker, date, time, text])

    # 4. Split into columns and save:
    # Instantiate the sentiment intensity analyzer
    vader = SentimentIntensityAnalyzer()
    # Set column names
    columns = ['ticker', 'date', 'time', 'headline']
    # Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
    parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)
    # Iterate through the headlines and get the polarity scores using vader
    scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()
    # Convert the 'scores' list of dicts into a DataFrame
    scores_df = pd.DataFrame(scores)
    # Join the DataFrames of the news and the list of dicts
    parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')
    # Convert the date column from string to datetime
    parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date
    parsed_and_scored_news.to_csv('~/LighthouseLabs-Final/Dataset/2. FinViz_Headline_Data/' + ticker + '_data_' + (datetime.today().strftime('%Y-%m-%d-%H')) + '.csv')

In [12]:
get_news('AAPL'), get_news('TSLA'), get_news('AMZN'), get_news('FB'), get_news('GOOG'), get_news('NFLX')

get_news('MSFT'), get_news('NVDA'), get_news('JNJ'), get_news('CVX'), get_news('PFE'), get_news('NKE'),get_news('GS')

print("Done!")

HTTPError: HTTP Error 503: Service Temporarily Unavailable

## 3. Historical Stock Data:

In [16]:
def stock_data(ticker):
    start_date = '2020-09-23'
    end_date = date.today()+timedelta(days=1)
    # 1. Request data:
    data = yf.download(ticker, 
                      start=start_date, 
                      end=end_date,
                      interval='30m', 
                      progress=False)
    # 2. Feature Engineering:
    data['Percent Price Change Within Period'] = ((data['Close'] - data['Open'])/data['Open'])*100
    # data['Change in Close Price'] = data['Close'] - data['Close'].shift(1)
    # data['Scaled Delta Close'] = data['Change in Close Price']/(data['Close'].mean())
    data['Scaled Volume'] = data['Volume']/data['Volume'].mean()
    data_SMA = data['Adj Close'].rolling(window=3).mean().shift(1)
    data['SMA(3)'] = data_SMA
    data['t+1'] = data['Adj Close'].shift(-1)
    data.reset_index(inplace=True)
    data['Datetime']=data['Datetime'].dt.tz_convert('America/Montreal').dt.tz_localize(None)
    data.drop(['Open','High','Low','Close'],axis=1,inplace=True)
    #3. Export data:
    f_name = ticker + "_data"
    data.to_csv('~/LighthouseLabs-Final/Dataset/1. Stock_Data/' + f_name + ".csv")
    print('Data saved!')
    # return data

In [17]:
stock_data('AAPL'), stock_data('GOOG'), stock_data('FB'), stock_data('AMZN'), stock_data('NFLX'), stock_data('TSLA')

stock_data('JNJ'), stock_data('MSFT'), stock_data('CVX'), stock_data('NKE'), stock_data('PFE'), stock_data('GS'), stock_data('NVDA')


Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!
Data saved!


(None, None, None, None, None, None, None)

In [19]:
# 2. Retrieve Headlines:
def get_news(ticker_code):
# 2.1 Define URL:
    finwiz_url = 'https://finviz.com/quote.ashx?t='
# 2.2 Requesting data:
    news_tables = {}
    tickers = [ticker_code]
    for ticker in tickers:
        url = finwiz_url + ticker
        req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
        response = urlopen(req)    
        # Read the contents of the file into 'html'
        html = BeautifulSoup(response,features="lxml")
        # Find 'news-table' in the Soup and load it into 'news_table'
        news_table = html.find(id='news-table')
        # Add the table to our dictionary
        news_tables[ticker] = news_table
# 2.3 Parsing news:
    parsed_news = []
    for file_name, news_table in news_tables.items():
        for x in news_table.findAll('tr'):
            text = x.a.get_text() 
            date_scrape = x.td.text.split()
            if len(date_scrape) == 1:
                time = date_scrape[0]
            else:
                date = date_scrape[0]
                time = date_scrape[1]
            ticker = file_name.split('_')[0]
            parsed_news.append([ticker, date, time, text])
# 2.4 Split into columns and save:
    vader = SentimentIntensityAnalyzer()
    columns = ['ticker', 'date', 'time', 'headline']
    processedHeadlines = pd.DataFrame(parsed_news, columns=columns)
    scores = processedHeadlines['headline'].apply(vader.polarity_scores).to_list()
    scores_df = pd.DataFrame(scores)
    processedHeadlines = processedHeadlines.join(scores_df, rsuffix='_right')
    processedHeadlines.insert(loc=1, column='timestamp', value=(pd.to_datetime(processedHeadlines['date'] + ' ' + processedHeadlines['time'])))
    processedHeadlines.drop(columns=['date','time'],axis=1,inplace=True)
    return processedHeadlines

In [20]:
get_news('AAPL')

,ticker,timestamp,headline,neg,neu,pos,compound
0,AAPL,2020-10-09 17:02:00,Is this a golden moment for independent films?,0.000,1.000,0.000,0.0000
1,AAPL,2020-10-09 16:35:00,Apple Starts Shipping Devices From Stores to S...,0.000,1.000,0.000,0.0000
2,AAPL,2020-10-09 16:25:00,"Apple Inc. stock rises Friday, outperforms market",0.000,1.000,0.000,0.0000
3,AAPL,2020-10-09 16:20:00,Apple COO Jeff Williams Sells Large Block of S...,0.266,0.734,0.000,-0.4404
4,AAPL,2020-10-09 15:38:00,Dow Jones Stocks To Buy And Watch In October 2...,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...
95,AAPL,2020-10-06 16:25:00,Apple Inc. stock underperforms Tuesday when co...,0.000,1.000,0.000,0.0000
96,AAPL,2020-10-06 16:23:00,Apple Set To Kick Off 5G Era With iPhone 12 La...,0.000,1.000,0.000,0.0000
97,AAPL,2020-10-06 16:05:00,Dow Slammed; Stocks Sell Off Hard After Trump ...,0.123,0.877,0.000,-0.1027
98,AAPL,2020-10-06 15:36:00,Big Tech shares drop amid reports House subcom...,0.137,0.719,0.144,0.0258


In [60]:
class VaderSentimentService:
    def __init__(self):
        self.vader = SentimentIntensityAnalyzer()
    
    def __call__(self, text):
        print(f'Text: {text}')
        return self.vader.polarity_scores(text)['compound']


In [89]:
vader = VaderSentimentService()


class TwitterScraper:
    def __init__(self):
        self.consumer_key = os.environ['consumer_key']
        self.consumer_secret = os.environ['consumer_secret']
        self.access_token = os.environ['twitter_access_token']
        self.access_token_secret = os.environ['twitter_access_secret']
        self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
        self.auth.set_access_token(self.access_token, self.access_token_secret)

    def __call__(self, hashtag_phrase, start_date=date.today(), end_date=date.today()+1):
        format_hashtag = '$' + hashtag_phrase

        api = tweepy.API(self.auth)
    
        scraped_tweets = []
        for tweet in tweepy.Cursor(
            api.search,
            q=format_hashtag + ' -filter:retweets',
            lang="en",
            tweet_mode='extended',
            since=start_date,
            until=end_date,
        ).values(): 
            scraped_tweets.append(
                {
                    'timestamp': tweet.created_at,
                    'tweet': self._clean_tweets(tweet.full_text.replace('\n',' ')),
                    'follow_count': tweet.user.followers_count,
                }
            )

        highest_follower_count = max(scraped_tweets, key=lambda x:x['price'])

        for tweet in scraped_tweets:
            tweet.update(vader(tweet['tweet']))
            tweet['scaled_follower_count'] = tweet['followers_count']/highest_follower_count
            tweet['compound_multiplier'] = tweet['compound'] * tweet['scaled_follower_count'] + 1

        # df.to_csv('~/LighthouseLabs-Final/' + hashtag_phrase + '_' + (datetime.today().strftime('%Y-%m-%d')) + '.csv')
        return scraped_tweets

    def _remove_pattern(self, input_txt, pattern):
        r = re.findall(pattern, input_txt)
        for i in r:
            input_txt = re.sub(i, '', input_txt)       
        return input_txt
        
    def _clean_tweets(self, tweets):
        #remove twitter Return handles (RT @xxx:)
        tweets = np.vectorize(self.remove_pattern)(tweets, "RT @[\w]*:") 
        #remove twitter handles (@xxx)
        tweets = np.vectorize(self.remove_pattern)(tweets, "@[\w]*")
        #remove URL links (httpxxx)
        tweets = np.vectorize(self.remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
        #remove special characters, numbers, punctuations (except for #)
        tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
        return tweets
        

TypeError: unsupported operand type(s) for +: 'datetime.date' and 'int'

In [3]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd

vader = VaderSentimentService()


class NewsHeadlineScraper:
    def __init__(self):
        self.finviz_url = 'https://finviz.com/quote.ashx?t='
    
    def __call__(self, ticker_symbols):
        for ticker in ticker_symbols:
            url = self.finviz_url + ticker.lower()
            req = Request(
                url=url,
                headers={'user-agent': 'Mozilla/5.0'},
            )
            response = urlopen(req).read()
            # Read the contents of the file into 'html'
            html = BeautifulSoup(response, "html.parser")
            # Find 'news-table' in the Soup and load it into 'news_table'
            news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
            news.columns = ['Date', 'News Headline']

            news['Sentiment'] = news['News Headline'].map(lambda x: vader(x))

        return news

In [4]:
finviz = NewsHeadlineScraper()
finviz('TSLA')

Text: Warner Bros. Discovery Stock Has Been Left for Dead -- Is It Time to Buy? Motley Fool
Text: AT&T COO Updates Shareholders at UBS Global TMT Conference Business Wire
Text: AT&T exec says this trend should give investors confidence MarketWatch
Text: Goldman: These Stocks Can Gain as Inflation Slows TheStreet.com
Text: 12 Best Communication Stocks To Buy Today Insider Monkey
Text: Goldman Sachs Plans Investing In FTX Hit Firms, Ticketmaster Slapped With Antitrust Lawsuit, Facebook Dating Gets Age Verification Tool: Top Stories Tuesday, Dec. 06 Benzinga
Text: AT&T To Settle SEC Lawsuit For Sharing Confidential Information With Analysts Benzinga
Text: A Tale of 2 Telecoms: Is AT&T or Verizon Stock the Better Buy? Motley Fool
Text: Is Verizon A Buy Or Sell As Holiday Season Promotions Heat Up? Investor's Business Daily
Text: Is T-Mobile A Buy Or Sell As Telecom Stock Out-Performs S&P 500? Investor's Business Daily
Text: HBO Max streaming service returns to Amazon Prime Video Reuters
Te

,Date,News Headline,Sentiment
0,Dec-07-22 09:45AM,Why This 1 Growth Stock Could Be a Great Addit...,0.7717
1,09:30AM,Why Agilent Technologies (A) is a Top Stock fo...,0.2023
2,Dec-06-22 09:00AM,"Is Trending Stock Agilent Technologies, Inc. (...",0.0000
3,Dec-05-22 08:00AM,Agilent Opens Customer Experience Center Highl...,0.1779
4,Nov-30-22 08:00AM,Agilent Attains Angel Level Sponsorship with M...,0.0000
...,...,...,...
95,Aug-17-22 04:31PM,Agilent Puts Together Commanding Beat That Ign...,-0.3612
96,02:18PM,"Agilent (A) Q3 Earnings Surpass Estimates, Rev...",0.0000
97,02:02PM,Stock Market Bounces Ahead Of Fed Minutes; Sma...,-0.3400
98,01:00AM,Agilent Technologies (A) Q3 2022 Earnings Call...,-0.4404


In [96]:
import yfinance as yf


class YahooStockData:
    def __call__(
        self,
        ticker_symbol,
        start_date=date.today(),
        end_date=date.today() + timedelta(days=1),
    ):
        # 1. Request data:
        data = yf.download(
            ticker_symbol, 
            start=start_date, 
            end=end_date,
            interval='30m', 
            progress=False,
        )
        # 2. Feature Engineering:
        data['Percent Price Change Within Period'] = ((data['Close'] - data['Open'])/data['Open'])*100
        # data['Change in Close Price'] = data['Close'] - data['Close'].shift(1)
        # data['Scaled Delta Close'] = data['Change in Close Price']/(data['Close'].mean())
        data['Scaled Volume'] = data['Volume']/data['Volume'].mean()
        data_SMA = data['Adj Close'].rolling(window=3).mean().shift(1)
        data['SMA(3)'] = data_SMA
        data['t+1'] = data['Adj Close'].shift(-1)
        data.reset_index(inplace=True)
        data['Datetime']=data['Datetime'].dt.tz_convert('America/Montreal').dt.tz_localize(None)
        data.drop(['Open','High','Low','Close'],axis=1,inplace=True)
        #3. Export data:
        f_name = ticker_symbol + "_data"
        data.to_csv('~/LighthouseLabs-Final/Dataset/1. Stock_Data/' + f_name + ".csv")
        print('Data saved!')
        # return data

In [98]:
yahoo = YahooStockData()

In [99]:
yahoo('TSLA')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\keato/LighthouseLabs-Final/Dataset/1. Stock_Data/TSLA_data.csv'

In [9]:
import os
import psycopg2

conn = psycopg2.connect(
        host="localhost",
        database="stock_market_predictor",
        user='postgres',
        # user=os.environ['DB_USERNAME'],
        password='Klait$08',
        # password=os.environ['DB_PASSWORD'],
    )

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command: this creates a new table
cur.execute('DROP TABLE IF EXISTS finviz;')
cur.execute(
    """
    CREATE TABLE finviz (
        id serial PRIMARY KEY,
        date varchar (150) NOT NULL,
        news_headline varchar (200) NOT NULL,
        sentiment integer NOT NULL
    )
    """
)

cur.execute('INSERT INTO finviz (date, news_headline, sentiment)'
            'VALUES (%s, %s, %s)',
            ('December , 2022',
             'Charles Dickens',
             489)
            )
conn.commit()

cur.close()
conn.close()
